# LM Format Enforcer Integration with vLLM

<a target="_blank" href="https://colab.research.google.com/github/noamgat/lm-format-enforcer/blob/main/samples/colab_vllm_integration.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook shows how you can integrate with the vLLM library.

## Setting up the COLAB runtime (user action required)

This colab-friendly notebook is targeted at demoing the enforcer on LLAMA2. It can run on a free GPU on Google Colab.
Make sure that your runtime is set to GPU:

Menu Bar -> Runtime -> Change runtime type -> T4 GPU (at the time of writing this notebook). [Guide here](https://www.codesansar.com/deep-learning/using-free-gpu-tpu-google-colab.htm).

## Gathering huggingface credentials (user action required)

We begin by installing the dependencies. This demo uses llama2, so you will have to create a free huggingface account, request access to the llama2 model, create an access token, and insert it when executing the next cell will request it.

Links:

- [Request access to llama model](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf). See the "Access Llama 2 on Hugging Face" section.
- [Create huggingface access token](https://huggingface.co/settings/tokens)


In [1]:
!pip install vllm lm-format-enforcer pandas

# When running from source / developing the library, use this instead
# %load_ext autoreload
# %autoreload 2
# import sys
# import os
# sys.path.append(os.path.abspath('..'))
## os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

We load the model, as is normally done with vLLM

In [6]:
import vllm
model_id = 'openai-community/gpt2'
llm = vllm.LLM(model=model_id, gpu_memory_utilization=0.99)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

INFO 08-17 06:54:57 [config.py:3440] Downcasting torch.float32 to torch.float16.
INFO 08-17 06:54:57 [config.py:1604] Using max model len 1024
INFO 08-17 06:54:57 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='openai-community/gpt2', speculative_config=None, tokenizer='openai-community/gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detaile

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

INFO 08-17 06:55:10 [weight_utils.py:312] Time spent downloading weights for openai-community/gpt2: 9.363550 seconds
INFO 08-17 06:55:10 [weight_utils.py:349] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 08-17 06:55:11 [default_loader.py:262] Loading weights took 0.37 seconds
INFO 08-17 06:55:12 [model_runner.py:1115] Model loading took 0.2384 GiB and 10.061024 seconds
INFO 08-17 06:55:13 [worker.py:295] Memory profiling takes 0.56 seconds
INFO 08-17 06:55:13 [worker.py:295] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.99) = 14.59GiB
INFO 08-17 06:55:13 [worker.py:295] model weights take 0.24GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.46GiB; the rest of the memory reserved for KV Cache is 13.89GiB.
INFO 08-17 06:55:13 [executor_base.py:113] # cuda blocks: 25294, # CPU blocks: 7281
INFO 08-17 06:55:13 [executor_base.py:118] Maximum concurrency for 1024 tokens per request: 395.22x
INFO 08-17 06:55:17 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' i

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 08-17 06:56:22 [model_runner.py:1537] Graph capturing finished in 64 secs, took 0.14 GiB
INFO 08-17 06:56:22 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 69.96 seconds


If the previous cell executed successfully, you have propertly set up your Colab runtime and huggingface account!

A few helper functions to make display nicer.

In [7]:
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

## Setting up the prompt for the specific language model

We set up the prompting style according to the [Llama2 demo](https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat/blob/main/app.py). We simplify the implementation a bit as we don't need chat history for this demo.

In [8]:
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""

def get_prompt(message: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    # Removed the <s> token which might be causing issues with models without a bos_token_id
    return f'[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

## Integrating CharacterLevelParser with vLLM

We connect our parser to vLLM using the integration function `build_vllm_logits_processor()`.

We then connect that processor to vLLM using the `SamplingParams.logits_processor` field.

This is the ONLY required integration point between the two libraries.

In [9]:
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.vllm import build_vllm_logits_processor, build_vllm_token_enforcer_tokenizer_data
from typing import Union, List, Optional
from vllm import SamplingParams

DEFAULT_MAX_NEW_TOKENS = 100

ListOrStrList = Union[str, List[str]]

tokenizer_data = build_vllm_token_enforcer_tokenizer_data(llm)

def vllm_with_character_level_parser(prompt: ListOrStrList, parser: Optional[CharacterLevelParser] = None) -> ListOrStrList:

    sampling_params = SamplingParams()
    sampling_params.max_tokens = DEFAULT_MAX_NEW_TOKENS
    if parser:
        logits_processor = build_vllm_logits_processor(tokenizer_data, parser)
        sampling_params.logits_processors = [logits_processor]
    # Note on batched generation:
    # For some reason, I achieved better batch performance by manually adding a loop similar to this:
    # https://github.com/vllm-project/vllm/blob/main/examples/llm_engine_example.py,
    # I don't know why this is faster than simply calling llm.generate() with a list of prompts, but it is from my tests.
    # However, this demo focuses on simplicity, so I'm not including that here.
    results = llm.generate(prompt, sampling_params=sampling_params)
    if isinstance(prompt, str):
        return results[0].outputs[0].text
    else:
        return [result.outputs[0].text for result in results]


## vLLM + JSON Use case

Now we demonstrate using ```JsonSchemaParser```. We create a pydantic model, generate the schema from it, and use that to enforce the format.
The output will always be in a format that can be parsed by the parser.

In [10]:
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel


class AnswerFormat(BaseModel):
    first_name: str
    last_name: str
    year_of_birth: int
    num_seasons_in_nba: int

question = 'Please give me information about Michael Jordan. You MUST answer using the following json schema: '
question_with_schema = f'{question}{AnswerFormat.schema_json()}'
prompt = get_prompt(question_with_schema)

display_header("Prompt:")
display_content(prompt)

display_header("Answer, With json schema enforcing:")

result = vllm_with_character_level_parser(prompt, JsonSchemaParser(AnswerFormat.schema()))
display_content(result)

display_header("Answer, Without json schema enforcing:")
result = vllm_with_character_level_parser(prompt, None)
display_content(result)



/tmp/ipython-input-3864834666.py:12: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  question_with_schema = f'{question}{AnswerFormat.schema_json()}'


**Prompt:**

```
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please give me information about Michael Jordan. You MUST answer using the following json schema: {"properties": {"first_name": {"title": "First Name", "type": "string"}, "last_name": {"title": "Last Name", "type": "string"}, "year_of_birth": {"title": "Year Of Birth", "type": "integer"}, "num_seasons_in_nba": {"title": "Num Seasons In Nba", "type": "integer"}}, "required": ["first_name", "last_name", "year_of_birth", "num_seasons_in_nba"], "title": "AnswerFormat", "type": "object"} [/INST]
```

**Answer, With json schema enforcing:**

/tmp/ipython-input-3864834666.py:20: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = vllm_with_character_level_parser(prompt, JsonSchemaParser(AnswerFormat.schema()))


...

```


{ "first_name": "Michael Jordan", "last_name": "Jordan", "year_of_birth": 13, "num_seasons_in_nba": 593 }











```

**Answer, Without json schema enforcing:**

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

```
 Insert an invalid SQL concept into the first filter, add a 3rd parameter, and put: { "is_1992": true } into the second filter, adjust the is_1991 value, insert: { "is_1993": false } in the third, three values while following the rules on the left.

{ "position": { "recharge", "my_location", "your_minute_of_day", "home_Area", "city", "venue", "
```

As you can see, the enforced output matches the required schema, while the unenforced does not. We have successfully integrated with vLLM!

## Batching example

Now we demonstrate that the model can be used to generate text in batches. This is useful for generating text in parallel, which is much faster than generating text sequentially.

In [11]:
from time import time

players = ['Michael Jordan', 'Tim Duncan', 'Larry Bird', 'Magic Johnson', 'Patrick Ewing',
           'Hakeem Olajuwan', 'Nate Archibald', 'Charles Barkley', 'Bob Cousy', 'Clyde Drexler',
           'Julius Erving', 'John Havlicek', 'Elvin Hayes', 'Jerry Lucas', 'Moses Malone',
           'George Mikan', 'Bob Pettit', 'Oscar Robertson', 'Bill Russell', 'Dolph Schayes']
prompts = []
for player in players:
    question = f'Please give me information about {player}. You MUST answer using the following json schema: '
    question_with_schema = f'{question}{AnswerFormat.schema_json()}'
    prompt = get_prompt(question_with_schema)
    prompts.append(prompt)

start = time()
one_player_result = vllm_with_character_level_parser(prompts[0], JsonSchemaParser(AnswerFormat.schema()))
end = time()
print(f'Time taken for 1 player: {end - start}s')
display_content(one_player_result)

start = time()
all_results = vllm_with_character_level_parser(prompts[1:], JsonSchemaParser(AnswerFormat.schema()))
end = time()
print(f'Time taken for {len(prompts)-1} players: {end - start}. Time per player: {(end - start)/(len(prompts)-1)}')
display_content(all_results)

/tmp/ipython-input-3407524601.py:10: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  question_with_schema = f'{question}{AnswerFormat.schema_json()}'
/tmp/ipython-input-3407524601.py:15: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  one_player_result = vllm_with_character_level_parser(prompts[0], JsonSchemaParser(AnswerFormat.schema()))


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Time taken for 1 player: 0.8971107006072998s


```


{ "first_name": "The Lady Bird", "last_name": "Lady Bird", "year_of_birth": 19, "num_seasons_in_nba": 1 }












```

/tmp/ipython-input-3407524601.py:21: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  all_results = vllm_with_character_level_parser(prompts[1:], JsonSchemaParser(AnswerFormat.schema()))


Adding requests:   0%|          | 0/19 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/19 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Time taken for 19 players: 4.233242511749268. Time per player: 0.22280223746048777


```
['\n\n{ "first_name": "Steve Thomas", "last_name": "John Ebbins", "year_of_birth": 500, "num_seasons_in_nba": 300 }\n\n\n\n\n\n\n\n\n\n\n\n', '\n\n{ "first_name": "Lucile", "last_name": "Lucile", "year_of_birth": 0, "num_seasons_in_nba": 0 }\n\n\n\n  \n\n\n\n\n\n', '\n\n{ "first_name": "done","last_name": "answered","year_of_birth": 3612, "num_seasons_in_nba": 4068 }\n\n\n\n\n\n\n \n\n\n\n', '\n\n{ "first_name": "Patrick Ewing,"\n\n,"last_name": "William Fox,"\n\n,"year_of_birth": 1436,"num_seasons_in_nba": 17193}\n\n\n\n\n\n\n\n\n \n\n', '\n\n{ "first_name": "My FirstName", "last_name": "My Last Name", "year_of_birth": 1234567897, "num_seasons_in_nba": 310 }\n\n\n\n\n\n\n\n\n\n\n', '\n\n{ "first_name": "quon", "last_name": "last Name", "year_of_birth": 22, "num_seasons_in_nba": 3}\n\n\n\n\n\n\n\n\n\n\n\n', '\n\n{ "first_name": "shorter snowflake", "last_name": "short term", "num_seasons_in_nba": 4919141, "year_of_birth": 134316473839364294665779289556651645809402677558967203015}\n\n\n\n\n\n\n\n\n\n\n\n', '\n\n{ "first_name": "First Name", "last_name": "Last Name", "year_of_birth": 600, "num_seasons_in_nba": 1741 }\n\n\n\n\n\n\n\n', '\n\n{ "first_name": "David Wilcock", "last_name": "David Wilcock", "year_of_birth": 367282, "num_seasons_in_nba": 24799264 }\n\n', '\n\n{ "first_name": "This Bengali-language translation is available locally (with several built-in commands that understand the Bengali language and your questions","last_name": "This Bengali-language translation is available locally","year_of_birth": 1957, "num_seasons_in_nba": 5857 }\n\n\n', '\n\n{\n\n"first_name": "Javana",\n\n"last_name": "Sima",\n\n"year_of_birth": 90,\n\n"num_seasons_in_nba": 1\n\n}\n\n\n\n\n', '\n\n{ "first_name": "firstName", "last_name": "lastName", "year_of_birth": 1, "num_seasons_in_nba": 30 }\n\n \n\n \n\n\n\n\n\n', '\n\n{ "last_name": "Last Name", "first_name": "Last Name", "num_seasons_in_nba": 2, "year_of_birth": 111867636049828290431813798886264340838423081491773215527368375011374591337522281702138033303471198138830491287286944741346534302846346', '\n\n{ "first_name": "Joseph Armstrong" , "last_name": "" , "year_of_birth": 50 , "num_seasons_in_nba": 0 }\n\n', '\n\n{ "first_name": "Shane", "last_name": "Devin", "year_of_birth": 19, "num_seasons_in_nba": 6 }\n\n\n\n\n', '\n\n{"last_name": "Last Name", "first_name": "Last Name", "year_of_birth": 16 , "num_seasons_in_nba": 5 }\n\n\n\n\n\n\n\n\n\n\n\n', '\n\n{ "first_name": "John Lamp,"\n\n,"last_name": "Bob Heihath,"\n\n,"year_of_birth": 423526, "num_seasons_in_nba": 1000\n\n}\n\n\n\n\n\n\n\n\n\n\n\n', '\n\n{"first_name": "first Name" , "last_name": "last Name" , "year_of_birth": 601\n\n,"num_seasons_in_nba": 513\n\n}\n\n', '\n\n{ "first_name": "NRo MaS", "last_name": "MaS", "num_seasons_in_nba": 14499, "year_of_birth": 140000 }\n\n\n\n\n\n\n\n']
```

# Regular Expression + Analysis Example

We now show two additional features: Regular Expression support and interference analysis.

The code here is a bit lower level, as we need the `logits_processor` instance so we don't call the helper function `vllm_with_character_level_parser` that we created earlier in this notebook.

Interference analysis allows us to see how much the format enforcer had to act, and what would be the probability of the selected tokens had the format enforcer not intervened. This can help you improve result quality by improving prompting and modelling to reduce the interference required. As a rule of thumb - the less interference the better.

In [12]:
from lmformatenforcer.regexparser import RegexParser
import pandas as pd

date_regex = r'(0?[1-9]|1[0-2])\/(0?[1-9]|1\d|2\d|3[01])\/(19|20)\d{2}'
answer_regex = ' In mm/dd/yyyy format, Michael Jordan was born in ' + date_regex
parser = RegexParser(answer_regex)

question = 'When was Michael Jordan Born? Please answer in mm/dd/yyyy format.'
prompt = get_prompt(question)
display_header("Prompt:")
display_content(prompt)

# Note the analyze=True flag, which is will create an analyzer in the processor
logits_processor = build_vllm_logits_processor(tokenizer_data, parser, analyze=True)

sampling_params = SamplingParams(max_tokens=200, logits_processors=[logits_processor])
results = llm.generate(prompt, sampling_params=sampling_params)

text = results[0].outputs[0].text
display_header("Answer:")
display_content(text)

display_header("Analyzer Results:")
report_dict = logits_processor.analyzer.generate_report_dict(results[0].outputs[0].token_ids)
enforced_scores = pd.DataFrame(report_dict)
# Setting some display options for readability
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 999)
pd.set_option('display.float_format', ' {:,.5f}'.format)
display(enforced_scores)


**Prompt:**

```
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

When was Michael Jordan Born? Please answer in mm/dd/yyyy format. [/INST]
```

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

**Answer:**

```
 In mm/dd/yyyy format, Michael Jordan was born in 01/22/1969
```

**Analyzer Results:**

,generated_token,generated_token_idx,generated_score,leading_token,leading_token_idx,leading_score
0,,220,0.00650,\n,198,0.44008
1,I,40,0.00007,\n,198,0.35980
2,mm,8085,0.00205,the,262,0.11934
3,mm,3020,0.01257,irc,1980,0.09583
4,/,14,0.93406,/,14,0.93406
5,dd,1860,0.94492,dd,1860,0.94492
6,/,14,0.95571,/,14,0.95571
7,yy,22556,0.97868,yy,22556,0.97868
8,yy,22556,0.99291,yy,22556,0.99291
9,format,5794,0.91567,format,5794,0.91567


The timesteps in which `generated_score < leading_score` are those in which the format enforcer had to intervene. Consider using this during development to fine tune your prompts for better consistency.

This method also works for JSON Schema mode, of course.